mkdir house-prices-starter
cd house-prices-starter
python -m venv venv
.venv\Scripts\activate     # On Windows

pip install pandas numpy scikit-learn kaggle

mkdir .kaggle
# Move the kaggle.json into .kaggle
# On Windows:
move path\to\kaggle.json .kaggle\kaggle.json

# Set environment variable:
set KAGGLE_CONFIG_DIR=.kaggle   # Windows
# export KAGGLE_CONFIG_DIR=.kaggle   # macOS/Linux

# Fix Kaggle API 401 Unauthorized (Windows PowerShell)

1. **Download API key** from [Kaggle Account](https://www.kaggle.com/account) → Create New API Token → save `kaggle.json`.

2. **Place `kaggle.json`** in `.kaggle` folder inside your project, e.g.  
`D:\Project\Kaggle\house-prices-starter\.kaggle\kaggle.json`

3. **Set env variable in PowerShell:**

```powershell
$env:KAGGLE_CONFIG_DIR = "D:\Project\Kaggle\house-prices-starter\.kaggle"

grep -qxF ".kaggle/kaggle.json" .gitignore || echo ".kaggle/kaggle.json" >> .gitignore

kaggle competitions download -c house-prices-advanced-regression-techniques
move .\house-prices-advanced-regression-techniques.zip .\data\
Remove-Item .\data\house-prices-advanced-regression-techniques.zip

In [ ]:
# Create folders if missing
$folders = @("notebooks", "scripts", "submissions", ".kaggle")

foreach ($folder in $folders) {
    if (-Not (Test-Path $folder)) {
        New-Item -ItemType Directory -Path $folder | Out-Null
        Write-Host "Created folder: $folder"
    } else {
        Write-Host "Folder already exists: $folder"
    }
}

# Create empty files if missing
$files = @("README.md", "requirements.txt")

foreach ($file in $files) {
    if (-Not (Test-Path $file)) {
        New-Item -ItemType File -Path $file | Out-Null
        Write-Host "Created file: $file"
    } else {
        Write-Host "File already exists: $file"
    }
}